In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

file_name="data/donnees_physicochimie.csv.zst"
pc_sample = pd.read_csv(file_name,nrows=1)
pc_list_cols = pc_sample.columns
pc_list_cat = pc_list_cols[pc_list_cols.str.startswith((
    'Lb','Nom','Mnemo',
    'Cd','Sym','Com'))]

pc_dict_cat = {col: 'category' for col in pc_list_cat}

physico_chimiques = pd.read_csv(
        "data/donnees_physicochimie.csv.zst",
        sep=',',
        dtype=pc_dict_cat,
        engine='c',
        escapechar='\\',
        parse_dates=[7],
        iterator=False)

hydrobio = pd.read_csv("data/donnees_hydrobio.csv.zst", sep=',')
df_stations = pd.read_csv("data/stations_hb.csv.zst", sep=';', escapechar='\\')

C:\Users\spoto\AppData\Local\Temp\ipykernel_10216\623196123.py:16: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  physico_chimiques = pd.read_csv(


In [7]:
df_stations.describe(include='all')

In [9]:

# 1. Filtrage des données (Nitrates)
nitrates = physico_chimiques[physico_chimiques['LbLongParamètre'] == 'Nitrates'].copy()
nitrates['DatePrel'] = pd.to_datetime(nitrates['DatePrel'])

# 2. Agrégation temporelle (moyenne mensuelle)
nitrates['Mois'] = nitrates['DatePrel'].dt.to_period('M')
nitrates_mensuels = nitrates.groupby(['CdStationMesureEauxSurface', 'Mois'])['RsAna'].mean().reset_index()
nitrates_mensuels['Mois'] = nitrates_mensuels['Mois'].dt.to_timestamp()
nitrates_mensuels = nitrates_mensuels.pivot(index='CdStationMesureEauxSurface', columns='Mois', values='RsAna')

# 3. Préparation des données pour le clustering
nitrates_scaled = StandardScaler().fit_transform(nitrates_mensuels.fillna(0)) # Gestion des valeurs manquantes

# 4. Clustering (exemple avec K-Means)
kmeans_bon = KMeans(n_clusters=3, random_state=42) # Exemple: 3 clusters pour les stations en bon état
kmeans_mauvais = KMeans(n_clusters=2, random_state=42) # Exemple: 2 clusters pour les stations en mauvais état

# ... Jointure avec les données hydrobiologiques et classification des stations ...
# Exemple: jointure et ajout d'une colonne 'EtatBiologique'
data = nitrates_mensuels.join(hydrobio.set_index('CdStationMesureEauxSurface')['EtatBiologique'], on='CdStationMesureEauxSurface')
data['EtatBiologique'] = data['EtatBiologique'].map({'Bon': 0, 'Mauvais': 1})

# ... Classification des stations en bon état ...
kmeans_bon.fit(nitrates_scaled)
data['ClusterBon'] = kmeans_bon.labels_

# ... Classification des stations en mauvais état ...
kmeans_mauvais.fit(nitrates_scaled)
data['ClusterMauvais'] = kmeans_mauvais.labels_

# 5. Visualisation des clusters
# Exemple: visualisation des clusters pour les stations en bon état
fig, ax = plt.subplots()
for cluster in data['ClusterBon'].unique():
    cluster_data = data[data['ClusterBon'] == cluster]
    ax.plot(cluster_data.dropna(axis=1).mean(), label=f'Cluster {cluster}')
ax.legend()
plt.show()




KeyError: 'EtatBiologique'